# Demo: RAIL Evaluation 

Contact: _Julia Gschwend_ ([julia@linea.gov.br](mailto:julia@linea.gov.br)), _Sam Schmidt, Alex Malz, Eric Charles_


The purpose of this notebook is to demonstrate the use of the metrics scripts to be used on the photo-$z$ PDF catalogs produced by the PZ working group. The first implementation of the _evaluation_ module is based on the refactoring of the algorithms used in [Schmidt et al. 2020](https://arxiv.org/pdf/2001.03621.pdf), available on Github repository [PZDC1paper](https://github.com/LSSTDESC/PZDC1paper). 

To run this code, you must install qp and have the notebook in the same directory as metrics.py. You must also install some run-of-the-mill Python packages: matplotlib, numpy, scipy, and skgof.

### Contents

* [Sample](#sample)
 - [Run FZBoost](#fzboost)
 - [Traditional validation plots](#old_valid)  
* [Metrics](#cdf_metrics)
 - [PIT](#pit) 
 - [QQ plot](#qq) 
 - [CDE loss](#cde_loss)  
* [Summary statistics](#summary_stats)
 - [KS](#ks) 
 - [CvM](#cvm) 
 - [AD](#ad) 
* [Summary](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown

#import warnings
#warnings.filterwarnings('ignore')

from sample import Sample
from metrics import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

<a class="anchor" id="sample"></a>

# Sample  


To compute the photo-z metrics of a given test sample, it is necessary to read the output of a photo-z code containing galaxies' photo-z PDFs. Let's use the toy data available in `tests/data/` (**test_dc2_training_9816.hdf5** and **test_dc2_validation_9816.hdf5**) and the configuration file available in `examples/configs/FZBoost.yaml` to generate a small samples of photo-z PDFs using the **FZBoost** algorithm available on RAIL's _estimation_ module.

<a class="anchor" id="fzboost"></a>
### Run FZBoost

Go to dir  `<your_path>/RAIL/examples/` and run the command:

`python main.py configs/FZBoost.yaml`

The photo-z output files (inputs for this notebook) will be writen at: 

`<your_path>/RAIL/examples/results/FZBoost/test_FZBoost.hdf5`. 

<font color='red'>The new RAIL's version will produce output of the codes as qp files rather than the old format hdf5 files (Sam's message on Slack about RAIL's issue#33). TO DO: update the read() function of class Sample </font>

In [ ]:
my_path = '/Users/julia/github/RAIL' # replace it by your path to RAIL's parent dir
pdfs_file = my_path + '/examples/results/FZBoost/test_FZBoost.hdf5'
ztrue_file = my_path + '/tests/data/test_dc2_validation_9816.hdf5'

Let's create a Sample object containing both the PDFs and true redshifts for each photo-z code.

In [ ]:
sample = Sample(pdfs_file, ztrue_file, code="FZBoost", name="toy data")
sample

In [ ]:
print(sample)

PDFs of 5 galaxies for illustration. The function `plot_pdfs` calls a _qp_ built-in plot function and returns the color codes of galaxies whose indexes are include in the list `gals`. The galaxies in the example were chosen arbitrarily to cover the sample's redshift space. The dashed lines represent their respective true redshifts.

In [ ]:
#gals = np.random.choice(len(ztrue), 5)
gals = [540, 2256, 12175, 17802, 19502]
colors = sample.plot_pdfs(gals)

<a class="anchor" id="old_valid"></a>
### Validation plots

Traditional validation plots. The point colors (optional) follow the same color code as the PDFs above. 

<font color='red'>TO DO: update the plots below to look like Figure 4 from CHIPPR's paper ([Malz & Hogg 2020](https://arxiv.org/pdf/2007.12178.pdf)). </font>


In [ ]:
sample.plot_old_valid(gals=gals, colors=colors)

<a class="anchor" id="cdf_metrics"></a>
# CDF-based metrics

The folowing metrics are computed based on the photo-z PDFs. Let's create a Metrics object to access the basic metrics (e.g., PIT outlier rate, defined below) and basic plots. It is the parent class of other particular metrics. 

Instantiating a Metrics object can take a bit long, depending on the sample size. 

In [ ]:
%%time
metrics = Metrics(sample)

<a class="anchor" id="pit"></a>

## PIT

The first metric we calculate is the Probability Integral Transform (PIT), which is the Cumulative Distribution Function (CDF) 

$$ \mathrm{CDF}(f, q)\ =\ \int_{-\infty}^{q}\ f(z)\ dz $$

evaluated at the galaxy's true redshift for every galaxy $i$ in the catalog.

$$ \mathrm{PIT}(p_{i}(z);\ z_{i})\ =\ \int_{-\infty}^{z^{true}_{i}}\ p_{i}(z)\ dz $$ 

For instance, the PIT values for the 5 PDFs shown above are:

In [ ]:
metrics.pit[gals]

#### PIT outlier rate

The PIT outlier rate is a global metric defined as the fraction of galaxies in the sample with extreme PIT values (PIT $<10^{-4}$ or PIT $>0.9999$). The lower and upper limits for considering a PIT as outlier are optional parameters set at the Metrics instantiation. 

In [ ]:
print(f"PIT outlier rate of this sample: {metrics.pit_out_rate:.4f}") 

## PIT-QQ plot

The histogram of PIT values is a useful tool for a qualitative assessment of PDFs quality. It shows whether the PDFs are:
* biased (tilted PIT histogram)
* under-dispersed (excess counts close to the boudaries 0 and 1)
* over-dispersed (lack of counts close the boudaries 0 and 1)
* well-calibrated (flat histogram)

Following the standards in DC1 paper, the PIT histogram is accompanied by the quantile-quantile (QQ), which can be used to compare qualitatively the PIT distribution obtained with the PDFs agaist the ideal case (uniform distribution). The closer the QQ plot is to the diagonal, the better is the PDFs calibration. 

In [ ]:
metrics.plot_pit_qq() #savefig=True)

By default, the function `plot_pit_qq` displays both PIT histogram and the QQ plots together. The title and label are retrieved from sample's attributed for sample name and the photo-z (if not informed as optional parameters). It is also possible to select one plot at a time. 

In [ ]:
metrics.plot_pit_qq(show_pit=False, show_pit_out_rate=False, title="QQ only")

It is possible ti set the number of bins in the PIT histigram. By default, it uses the same number of quantiles, which is an attribute of the metrics object.

In [ ]:
metrics.n_quant

In [ ]:
metrics.plot_pit_qq(show_qq=False, title="PIT only", bins=30)

The black horizontal line represents the ideal case where the PIT histogram would behave as a uniform distribution U(0,1). 

Let's explore a different number of quantiles. By default, a Metrics object is instantiated with parameter `n_quant` = 100 percentiles. To redefine the number of quantiles, let's create a new Metrics object.

In [ ]:
metrics_20 = Metrics(sample, n_quant=20)

In [ ]:
metrics_20.plot_pit_qq(title="N$_{quantiles}$=20")

<a class="anchor" id="cde_loss"></a>
## CDE Loss

Next we can calculate the Conditional Density Estimate (CDE) loss described in [Izbicki & Lee, 2017 (arXiv:1704.08095)](https://arxiv.org/abs/1704.08095). 

$$ \int \int ((p(z \mid x) - \hat{p}(z \mid x))^{2} dz dP(x) $$

which extends L2 density estimation loss to conditional density estimation.  We can estimate this quantity (up to an unknown additive constant which depends on the true conditional densities) from data as

$$ \frac{1}{n} \sum_{i=1}^{n} \int \hat{p}^{2}(z \mid x_{i}) dz - \frac{2}{n} \sum_{i=1}^{n} \hat{p}(z_{i} \mid x_{i}) $$

<font color='red'> TO DO: update equations to match the paper </font>

In [ ]:
print(f"CDE loss of this sample: {metrics.cde_loss:.2f}") 

<a class="anchor" id="summary_stats"></a>
# Summary statistics of CDF-based metrics

To evaluate globally the quality of PDFs estimates, we compute a number of statistical tests to compare the empirical distributions of PIT values with the theoretically perfect  distribution, U(0,1).  

The individual metrics are implemented as independent classes receiving the PIT array as input. They can be accessed separately or incorporated to the generic metrics object to compose the sumary of all metrics.

<a class="anchor" id="ks"></a>
### Kolmogorov-Smirnov  

We start with the Kolmogorov-Smirnov (KS) statistic test, which is the maximum difference between the empirical and the expected distributions:

$$
\mathrm{KS} \equiv \max_{z} \Big( \left| \ \mathrm{CDF} \small[ \hat{f}, z \small] - \mathrm{CDF} \small[ \tilde{f}, z \small] \  \right| \Big)
$$

so the smaller value of KS the closer the PIT distribution is to be uniform. The KS metric inherits from scipy.stats.kstest method, returning the KS maximum distance and the p-value. 

In [ ]:
ks = KS(metrics._pit)

In [ ]:
ks.stat

In [ ]:
ks.pvalue

<a class="anchor" id="cvm"></a>
### Cramer-von Mises

Similarly, we calculate the Cramer-von Mises (CvM) test, a variant of the KS statistic defined as the mean-square difference between the CDFs of an empirical PDF and the true PDFs:

$$ \mathrm{CvM}^2 \equiv \int_{-\infty}^{\infty} \Big( \mathrm{CDF} \small[ \hat{f}, z \small] \ - \ \mathrm{CDF} \small[ \tilde{f}, z \small] \Big)^{2} \mathrm{dCDF}(\tilde{f}, z) $$ 


on the distribution of PIT values, which should be uniform if the PDFs are perfect.

In [ ]:
cvm = CvM(metrics._pit)

In [ ]:
cvm.stat

In [ ]:
cvm.pvalue

<a class="anchor" id="ad"></a>
### Anderson-Darling 

Another variation of the KS statistic is the Anderson-Darling (AD) test, a weighted mean-squared difference featuring enhanced sensitivity to discrepancies in the tails of the distribution. 

$$ \mathrm{AD}^2 \equiv N_{tot} \int_{-\infty}^{\infty} \frac{\big( \mathrm{CDF} \small[ \hat{f}, z \small] \ - \ \mathrm{CDF} \small[ \tilde{f}, z \small] \big)^{2}}{\mathrm{CDF} \small[ \hat{f}, z \small] \big( 1 \ - \ \mathrm{CDF} \small[ \tilde{f}, z \small] \big)}\mathrm{dCDF}(\tilde{f}, z) $$ 

where catastrophic outliers (default is PIT<0.01 or PIT>0.99) are removed for preserving numerical instability. Note that these values are not the same used to define the PIT outlier rate above (0.0001, and 0.9999). 

The AD metric also inherits from scipy.stats library, returning three quantities: the AD stat value, a list of significance levels , and a list of critical values, and regarding those significance levels.

<font color='red'> QUESTION: scipy.stats.anderson method supports the following distributions as reference: {‘norm’, ‘expon’, ‘logistic’, ‘gumbel’, ‘gumbel_l’, ‘gumbel_r’, ‘extreme1’}. Apparently there is no way to compare the PIT distribution with a uniform distribution using this method. I'm using the default option ('norm'), but I think it's wrong.</font>

In [ ]:
ad = AD(metrics._pit)

In [ ]:
ad.stat

In [ ]:
ad.significance_levels

In [ ]:
ad.critical_values

<font color='red'> TO DO: add sentence with interpretation to the numbers above</font>

<a class="anchor" id="summary"></a>
# Summary

In [ ]:
metrics_table = metrics.markdown_summary() 
Markdown(metrics_table)

<font color='red'> TO DO: Metrics plot (Figure 3 from DC1 Paper: [Schmidt et al. 2020](https://arxiv.org/pdf/2001.03621.pdf) )</font>